# Modelo de aprendizagem

Este é um modelo de aprendizagem básico para o aplicativo proposto. Para seguir de uma forma didática, usarei o formato de notebook para facilitar o entendimento de todos em cada passo.

Inicialmente, irei explorar os dados e explicá-los, assim podemos entender melhor como podemos criar um modelo de aprendizagem para eles.

Os dados foram gerados, mas não foram criados de forma aleatória. A ideia é trabalhar com dados de mapas astrais obtidos a partir da data e hora de nascimento e local de nascimento. Existem algumas bases na internet com mapas de pessoas famosas mas como esses dados não são disponibilizados de forma pura, a tarefa de extraí-los se torna complicada já que a maioria possui mecanismos que bloqueiam o IP de quem tenta usar técnicas de scraping, ou crawling. Nesse projeto existe um esboço de um código para o scraping de um desses sites, porém o código não pode ser concluído pois meu IP foi banido do site. Além disso, precisamos de dados localizados, isto é, já que a localização é levada em conta, precisamos de dados do Brasil primeiramente.

Por esses motivos, criei o seguinte mecanismo para gerar dados reais. Construi uma aplicação capaz de gerar os parâmetros utilizados para criar os mapas, o que nos interessa, assim, pude gerar datas aleatórias e localizações aleatórias dentre algumas cidades do Brasil. Para adicionar um certo ruído na base, adicionei cidades de outros países. Esse mecanismo pode ser acessado também no código fonte do projeto.


In [22]:
import pandas as pd
import numpy as np
import random as rd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

Agora vamos explorar a base de dados. Primeiro vamos carregar o arquivo.

In [45]:
df = pd.read_csv("../Generator/data.csv")
df.head()

,record_num,m_birth_date,m_birth_hour,m_born_lat,m_born_long,m_sign,m_sun,m_sun_speed,m_sun_house,m_moon,...,f_house8_speed,f_house9,f_house9_speed,f_MC,f_MC_speed,f_house11,f_house11_speed,f_house12,f_house12_speed,success_rate
0,1,07.05.1962,11:55:57,-15.80,-47.86667,TAURUS,46.443365,0.967946,4.000000,89.706202,...,5.189859,336.500871,5.189859,9.749399,5.189859,40.201877,5.189859,67.294476,5.189859,75
1,2,09.02.1942,12:50:25,48.85,-0.11667,AQUARIUS,320.149983,1.012209,9.630953,242.214946,...,11.936511,151.582030,11.936511,182.224710,11.936511,216.324373,11.936511,250.212250,11.936511,90
2,3,15.07.1992,20:08:59,-20.80,-49.38333,CANCER,113.559767,0.953598,6.000000,305.012308,...,3.263275,238.229917,3.263275,269.979165,3.263275,301.728243,3.263275,332.171993,3.263275,60
3,4,29.12.1986,00:25:55,-15.60,23333.00000,CAPRICORN,276.979663,1.019540,6.000000,246.896129,...,8.579950,51.847728,8.579950,85.556608,8.579950,119.450473,8.579950,150.054832,8.579950,85
4,5,09.06.1965,18:53:46,48.85,-0.11667,GEMINI,78.700647,0.955866,10.000000,209.374219,...,3.070671,27.079885,3.070671,56.444370,3.070671,82.476854,3.070671,108.195782,3.070671,90


A tabela acima contém um relacionamento de dois dados. Essa associação foi feita através de signos e para isso usei uma referência. Encontrei o seguinte site com uma tabela com porcentagens de compatibilidade: http://www.findhoroscope.com/love-compatibility-chart/ que encaixou perfeitamente para o uso, já que a ideia é encontrar se um relacionamento será bem sucedido ou não.

Com essa referência, podemos associar o signo e dar uma probabilidade. Mas podemos ajustar a probabilidade usando outros signos, como, por exemplo, o signo relacionado à lua, usar a mesma tabela e adicionar um ajuste. Claro que, esse tipo de ajuste precisa ser feito com estudo sobre as influências de cada parâmetro na compatibilidade, por isso, opto por associar apenas com o signo principal (sol). 

Agora, vamos converter os signos para dados numéricos, para que nosso modelo consiga entender. Além disso, vamos remover as datas e a latitude e longitude da localidade já que isso também não interessa.

In [46]:
def changeSign(row):
    sign_nums = {
        "ARIES": 0,
        "TAURUS": 1,
        "GEMINI": 2,
        "CANCER": 3,
        "LEO": 4,
        "VIRGO": 5,
        "LIBRA": 6,
        "SCORPIO": 7,
        "SAGITTARIUS": 8,
        "CAPRICORN": 9,
        "AQUARIUS": 10,
        "PISCES": 11
    }
    
    row["m_sign"] = sign_nums.get(row["m_sign"])
    row["f_sign"] = sign_nums.get(row["f_sign"])
    return row

df = df.apply(changeSign, axis=1)

df = df.drop(["m_birth_date", "m_birth_hour", "m_born_lat", "m_born_long", "f_birth_date", "f_birth_hour", "f_born_lat", "f_born_long"], axis=1)

df.head()

,record_num,m_sign,m_sun,m_sun_speed,m_sun_house,m_moon,m_moon_speed,m_moon_house,m_mercury,m_mercury_speed,...,f_house8_speed,f_house9,f_house9_speed,f_MC,f_MC_speed,f_house11,f_house11_speed,f_house12,f_house12_speed,success_rate
0,1,1,46.443365,0.967946,4.000000,89.706202,13.325487,1.0,66.592284,1.417432,...,5.189859,336.500871,5.189859,9.749399,5.189859,40.201877,5.189859,67.294476,5.189859,75
1,2,10,320.149983,1.012209,9.630953,242.214946,14.256026,10.0,321.105641,-1.168389,...,11.936511,151.582030,11.936511,182.224710,11.936511,216.324373,11.936511,250.212250,11.936511,90
2,3,3,113.559767,0.953598,6.000000,305.012308,11.891468,1.0,136.770398,0.330993,...,3.263275,238.229917,3.263275,269.979165,3.263275,301.728243,3.263275,332.171993,3.263275,60
3,4,9,276.979663,1.019540,6.000000,246.896129,14.991231,1.0,268.526962,1.553418,...,8.579950,51.847728,8.579950,85.556608,8.579950,119.450473,8.579950,150.054832,8.579950,85
4,5,2,78.700647,0.955866,10.000000,209.374219,12.895113,2.0,76.207259,2.193602,...,3.070671,27.079885,3.070671,56.444370,3.070671,82.476854,3.070671,108.195782,3.070671,90


Temos uma tabela com 149 features numéricas. Aqui irei apresentar um modelo de Random Forest para estimar a probabilidade de um relacionamento dar certo ou não. Vamos definir que um relacionado que deu certo é aquele que tem probabilidade maior que 75% de acontecer.

In [47]:
result_df = df[["record_num","success_rate"]].copy()
result_df["success"] = result_df.apply(lambda row: 1 if row["success_rate"] >= 75 else 0, axis=1)
result_df.head(20)
    

,record_num,success_rate,success
0,1,75,1
1,2,90,1
2,3,60,0
3,4,85,1
4,5,90,1
5,6,85,1
6,7,80,1
7,8,85,1
8,9,45,0
9,10,70,0


Agora vamos para o modelo

In [49]:
df = df.fillna(0)
clf = RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=3)
values = df.drop(["record_num", "success_rate"], axis=1).values
result = result_df["success"].values

scores = cross_val_score(clf, values, result, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.65 (+/- 0.04)


Até agora nosso modelo não está bom, precisamos melhorar e muito aqui. Uma ideia é aplicar o XGBoost e ver qual resultado ganhamos.